# Ссылка на данные

https://disk.yandex.ru/d/njZFAs-9_2b_lA

![schema_data](schema_data.png)

## Первичный осмотр данных

In [1]:
import pandas as pd
import numpy as np
import warnings
pd.set_option('display.max_columns', None)
warnings.filterwarnings('ignore')
df_sellers = pd.read_csv('sellers.csv',index_col='Unnamed: 0')
df_orders_item = pd.read_csv('orders_items.csv',index_col='Unnamed: 0')
df_final = df_sellers.merge(df_orders_item, how = 'right', on = 'seller_id')
df_final.drop_duplicates(inplace = True)
df_products = pd.read_csv('products.csv')
df_final = df_final.merge(df_products, how = 'left', on = 'product_id')
df_category_name = pd.read_csv('product_category_name_translation.csv',index_col='Unnamed: 0')
df_final = df_final.merge(df_category_name, how = 'left', on='product_category_name')
df_orders = pd.read_csv('orders.csv')
df_payments = pd.read_csv('order_payments.csv',index_col='Unnamed: 0')
df_final_2 = df_orders.merge(df_payments, on = 'order_id')
df_customers = pd.read_csv('customers.csv')
df_final_2 = df_final_2.merge(df_customers, on = 'customer_id')
df_order_reviews = pd.read_csv('order_reviews.csv',index_col='Unnamed: 0')
df_final_2 = df_final_2.merge(df_order_reviews, on = 'order_id', how = 'left')
df_itog = df_final_2.merge(df_final, on = 'order_id', how = 'left')
df_geolocation = pd.read_csv('geolocation.csv',index_col='Unnamed: 0')
df_geolocation.drop(['geolocation_lat','geolocation_lng'],axis = 1, inplace = True)
df_geolocation.drop_duplicates(inplace = True)
df_geolocation.reset_index(drop = True, inplace = True)
df_geolocation_seller = df_geolocation.copy()
df_geolocation_seller.columns =['seller_'+ col for col in df_geolocation_seller.columns]
df_geolocation_customers = df_geolocation.copy()
df_geolocation_customers.columns =['customer_'+ col for col in df_geolocation_customers.columns]
df_itog = df_itog.merge(df_geolocation_seller , how = 'left', left_on='seller_zip_code_prefix' , right_on='seller_geolocation_zip_code_prefix')
df_itog = df_itog.merge(df_geolocation_customers , how = 'left', left_on='customer_zip_code_prefix' , right_on='customer_geolocation_zip_code_prefix')
df_itog.drop_duplicates(inplace = True)
df_itog.reset_index(drop = True, inplace = True)

In [2]:
df_itog.to_csv('final_df_v2.csv',index = False)

In [3]:
dtypes = {
    'payment_sequential': 'int16',
    'payment_installments': 'int16',
    'payment_value': 'float32',
    'customer_zip_code_prefix': 'int32',
    'seller_zip_code_prefix': 'Int32',
    'order_item_id': 'Int16',
    'price': 'float32',
    'freight_value': 'float32',
    'price.1': 'float32',
    'freight_value.1': 'float32',
    'product_name_lenght': 'Int16',
    'product_description_lenght': 'Int16',
    'product_photos_qty': 'Int16',
    'product_weight_g': 'float32',
    'product_length_cm': 'float32',
    'product_height_cm': 'float32',
    'product_width_cm': 'float32',
    'customer_geolocation_zip_code_prefix': 'Int32',
    'order_status': 'category',
    'payment_type': 'category',
    'customer_state': 'category',
    'seller_state': 'category',
    'product_category_name': 'category',
    'product_category_name_english': 'category',
    'customer_geolocation_state': 'category',
    'customer_city': 'category',
    'seller_city': 'category',
    'seller_geolocation_city': 'category',
    'customer_geolocation_city': 'category'
}
parse_dates = [
    'order_purchase_timestamp', 'order_approved_at',
    'order_delivered_carrier_date', 'order_delivered_customer_date',
    'order_estimated_delivery_date', 'review_creation_date',
    'review_answer_timestamp', 'shipping_limit_date', 'shipping_limit_date.1'
]
df_itog = pd.read_csv("final_df_v2.csv", dtype=dtypes, parse_dates=parse_dates)
df_itog.drop(['shipping_limit_date.1','price.1','freight_value.1'],inplace = True, axis = 1)
df_itog.drop_duplicates(subset=[col for col in df_itog.columns if col not in ['customer_geolocation_city','seller_geolocation_city']], 
                        inplace = True)
df_itog.reset_index(inplace = True, drop = True)
df_itog.to_csv('final_df_v2.csv',index = False)